### Import needed libraries

In [1]:
# library provided by EPOCH for reading .sdf output files into Python
import sdf 

# python plotting library similar to MATLAB(TM)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# signal processing, needed for convolution
from scipy import signal

# Python array manipulation
import numpy as np

# various math functions 
import math as m

In [ ]:
# show plots in the notebook
%matplotlib inline

### Define useful functions

In [ ]:
from collections import OrderedDict

In [ ]:
def get_sdf_files(path):
    r"""Given a ``path``, it returns an ordered dictionary containing all the .sdf files in that path, 
    in the form {#### : /../../####.sdf}"""
    
    sdfs = glob.glob(os.path.join(path, '*.sdf')) 
    sdf_dict = {int(sdf.split('.')[0][-4:]):sdf for sdf in sdfs}
    sdf_ord_dict = OrderedDict(sorted(sdf_dict.items(), key=lambda t: t[1])) 
    
    return sdf_ord_dict

In [ ]:
def center_linspace(space):
    r"""
    
    Parameters
    ----------
    space : ndarray
        A one-dimensional array, of linearly spaced numbers. 
    
    Returns
    -------
    out : ndarray
        Also an array of the same spacing, this time centered around 0.
    """
    
    assert np.ndim(space) == 1
    size = space.size
    first = space[0]
    last = space[-1]
    length = last - first
    end = length/2
    
    _, step = np.linspace(first, last, size, retstep=True)
    
    out, out_step = np.linspace(-end, end, size, retstep=True) 
    assert m.isclose(out_step, step)
    return out

In [ ]:
def list_sdf_variables(data):
    r"""Lists all the quantities from the .sdf file.
    
    Parameters
    ----------
    data : ``sdf.Blocklist``
        The results of calling sdf.read on an .sdf file.
    """
    dct = data.__dict__
    for key in sorted(dct):
        try:
            val = dct[key]
            print('{} {} {}'.format(key, type(val),
                  np.array2string(np.array(val.dims), separator=', ')))
        except:
            pass

In [ ]:
def colorbar(mappable):
    r"""Constructs a scaled colorbar for a given plot.
    
    Parameters
    ----------
    mappable : The Image, ContourSet, etc. to which the colorbar applies.
    """
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax)

In [ ]:
def gauss_kern_nd(n, sizes):
    r"""Constructs a Gaussian kernel in any number of dimensions.
    
    Parameters
    ----------
    n : int
        The number of dimensions. Must be 1 or above.
    sizes : list
        The sizes along the various dimensions, ie. in 3D this would be [size_x, size_y, size_x]. The length of
        this list should be either 1 or n. If it only contains one element, eg. [size], it is assumed that
        size_x = size_y = size_x = size.
    
    Returns
    -------
    g : ndarray
        The kernel, with dimensions (in 3d) (2*size_x+1, 2*size_y+1, 2*size_z+1).
    """
    assert n > 0, 'at least 1d required'
    no_sizes = len(sizes)
    assert no_sizes == 1 or no_sizes == n, 'either give one size or all of them'
    
    for i in range(n - no_sizes):
        sizes.append(sizes[0])
        
    slices = tuple(slice(-size,size+1,None) for size in sizes)
    
    XXX = np.mgrid[slices]
    
    g = np.ones(XXX.shape[1:], dtype=np.float64)
    
    for X, size in zip(XXX, sizes):
        g = g * np.exp(-X**2/size)
        
    return g / g.sum()

In [ ]:
def smooth(data, n, sizes):
    r"""Smoothens the input by performing a convolution with a Gaussian kernel.
    
    Parameters
    ----------
    data : ndarray
        Input data, with ``n`` dimensions.
    n : int
        The number of dimensions. Must be 1 or above.
    sizes : list
        The sizes along the various dimensions, ie. in 3D this would be [size_x, size_y, size_x]. The length of
        this list should be either 1 or n. If it only contains one element, eg. [size], it is assumed that
        size_x = size_y = size_x = size.
        
    Returns
    -------
    out : ndarray
        The smoothed input, of shape ```data.shape - 2 * sizes```.
    """
    
    g = gauss_kern_nd(n, sizes)
    out = signal.convolve(data, g, mode='valid')
    return(out)

bash console commands can be run inside the notebook, eg.

The following code will download the data needed for this notebook automatically
using `curl`. It may take some time (the archive is 1.2 GB), so please wait when
the kernel is busy. You will need to set `download_datasets` to `True` before
using it.

In [ ]:
download_datasets = False
if download_datasets:
    !curl -sSO https://ndownloader.figshare.com/articles/5545165/versions/1
    print ("Downloaded the EPOCH data from figshare.")
    !unzip 1 
    
    print ("All done!")

In [ ]:
# these are the .sdf files used in the notebook. they must be present in the same folder when the notebook is run
!ls -lsa *.sdf

In [ ]:
# these are the corresponding EPOCH input decks
!ls -lsa *.deck

And even latex can be used:
$$E^2 = (pc)^2 + (mc^2)^2$$

# 1D case

For the 1D case, we just use `numpy` and `matplolib` for low-level plotting and
data analysis.

In [ ]:
# load the sdf file produced by EPOCH
# ignore the warning
fname = '1d.sdf'
d = sdf.read(fname)
fields = d.__dict__

In [ ]:
# assign separate variables to the various quantities we want to look at
grid = fields['Grid_Grid_mid']
nele = fields['Derived_Number_Density_ele']
ex = fields['Electric_Field_Ex']
ey = fields['Electric_Field_Ey']

In [ ]:
# the .data attribute contains the raw data we need to plot
(x,) = grid.data
# convert to micrometers
x = x*1e6
rho = nele.data

In [ ]:
# define size of Gaussian kernel used for noise reduction
kern_size = 100
rho_smooth = smooth(rho, 1, [kern_size])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,3])

axes[0].plot(x, rho)
axes[1].plot(x[kern_size:-kern_size], rho_smooth)

axes[0].set_title('raw data')
axes[1].set_title('after convolution with Gaussian kernel')

for ax in axes:
    # the labels and units were contained in the .sdf file
    ax.set_xlabel(nele.grid_mid.labels[0] + r' $(\mu m)$', labelpad=-1)
    ax.set_ylabel(nele.name + r' $(' + nele.units + r')$');

plt.tight_layout(h_pad=1)

# saves the generated figure to disk in the same folder that the notebook is ran from
fig.savefig('1d_rho.png')

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,3])

for ax, ef in zip(axes, (ex.data, ey.data)):
    ax.plot(x, ef)

for ax, electric in zip(axes, (ex, ey)):
    ax.set_xlabel(electric.grid_mid.labels[0] + r' $(\mu m)$', labelpad=-1)
    ax.set_ylabel(electric.name + r' $(' + electric.units + r')$');

fig.suptitle('components of the electric field')
plt.tight_layout(h_pad=1)

# 2D case

Now we analythe the results from a simple 2D simulation. We use the higher-level
library [`holoviews`](http://holoviews.org) for interactive plotting.

In [ ]:
fname = '2d.sdf'
d = sdf.read(fname)
fields = d.__dict__

In [ ]:
grid = fields['Grid_Grid_mid']
nele = fields['Derived_Number_Density_ele']
ex = fields['Electric_Field_Ex']
ey = fields['Electric_Field_Ey']

In [ ]:
(x, y) = grid.data
# convert to micrometers
x = x*1e+6
y = y*1e+6

In [ ]:
# get boundaries of simulation box
x_min = np.min(x); x_max = np.max(x)
y_min = np.min(y); y_max = np.max(y)

In [ ]:
# note we transpose the raw data 
rho = nele.data.T

# we convolute with a Gaussian kernel of size (30, 30) to reduce the noise
kern_size = 30
rho_smooth = smooth(nele.data.T, 2, [kern_size])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,4])

imgs = []
for ax, r in zip(axes, (rho, rho_smooth)):
    img = ax.imshow(r, origin='lower', interpolation='none',
                    extent=np.array([x_min, x_max, y_min, y_max]),
                    aspect='auto')
    imgs.append(img)

axes[0].set_title('raw data')
axes[1].set_title('after convolution with Gaussian kernel')

for ax in axes:
    for axis, label in zip([ax.xaxis, ax.yaxis], nele.grid_mid.labels):
        axis.set(label_text=label + r' $(\mu m)$')

# add colorbars to the plots
cbars = [colorbar(img) for img in imgs]

for cbar in cbars:
    cbar.set_label(nele.name + r' $(' + nele.units + r')$')

plt.tight_layout(h_pad=1)

In [ ]:
# electric field, x and y components
efx = ex.data.T
efy = ey.data.T

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,4])

imgs = []
for ax, ef in zip(axes, (efx, efy)):
    img = ax.imshow(ef, origin='lower', interpolation='none',
                    extent=np.array([x_min, x_max, y_min, y_max]),
                    aspect='auto')
    imgs.append(img)

for ax, ef in zip(axes, (ex,ey)):
    for axis, label in zip([ax.xaxis, ax.yaxis], ef.grid_mid.labels):
        axis.set(label_text=label + r' $(\mu m)$')

# add colorbars to the plots
cbars = [colorbar(img) for img in imgs]

for cbar, ef in zip(cbars, (ex, ey)):
    cbar.set_label(ef.name + r' $(' + ef.units + r')$')

plt.tight_layout(h_pad=1)

In [ ]:
# integrate out the x dependence
e_sum_x = np.sum(np.abs(efy), axis=1)

In [ ]:
# we get a Gaussian
fig, ax = plt.subplots()
ax.plot(y, e_sum_x)
ax.set_xlabel(ey.grid_mid.labels[1] + r' $(\mu m)$', labelpad=0)
ax.set_ylabel(r'$\sum_{x} |E_y(x,y)|$ [V/m]');

In [ ]:
# interactive visualization library
import holoviews as hv
hv.extension('matplotlib')

In [ ]:
# set global plotting and style options for holoviews
%opts Image (cmap='viridis') [colorbar=True] Bounds (color='red') HLine (color='red') VLine (color='red')

We want to look at the electron density. We first define its dimensions, along
with the corresponding units.

In [ ]:
xdim = hv.Dimension('x', label='x', unit=r'$\mu$'+'m')
ydim = hv.Dimension('y', label='y', unit=r'$\mu$'+'m')
zdim = hv.Dimension('z', label=nele.name, unit=r'$'+nele.units+r'$')

In [ ]:
# this are the ranges of the axes after the convolution
k = kern_size
xrange = x[k:-k]
yrange = y[k:-k]

In [ ]:
# we load the smoothed electron density into an Image object
img = hv.Image((xrange, yrange, rho_smooth), datatype=['grid'], 
                 kdims=[xdim, ydim], vdims=[zdim])

In [ ]:
# we want to focus on a particular region of the density, defined by these coordinates
x1 = xrange[60]; x2 = xrange[-500]
y1 = -20; y2 = 20

In [ ]:
# we plot the full image and the region of interest side by side

In [ ]:
%%output size=150
img_box = img[x1:x2, y1:y2]
box = hv.Bounds((x1,y1,x2,y2))
img*box + img_box

Now we can define a slice along the `y` axis and interactively move it.

In [ ]:
x_slice = {x : img_box * hv.VLine(x=x) + img_box.sample(x=x) for x in np.linspace(x1, x2, 50)}

In [ ]:
hv.HoloMap(x_slice, kdims=['x']).collate()

And of course, we can do the same along `x`.

In [ ]:
y_slice = {y : img_box * hv.HLine(y=y) + img_box.sample(y=y) for y in np.linspace(y1, y2, 50)}

In [ ]:
hv.HoloMap(y_slice, kdims=['y']).collate()

And we can plot the region of interest, together with the two slices, in one
line. Notice `holoviews` automatically knows how to label the plots.

In [ ]:
%%output size=150
img_box * hv.HLine(y=0.) * hv.VLine(x=787) + img_box.sample(y=0.) + img_box.sample(x=787)

# 3D case

In [ ]:
import os, glob

In [ ]:
root = '/home/berceanu/Development/pic/ming_jobs'
folder = 'ep_III3D1'
rf = os.path.join(root, folder)

sdfs = glob.glob(os.path.join(rf, '*.sdf')) 
sdf_file = {int(sdf.split('.')[0][-4:]):sdf for sdf in sdfs}

fname = sdf_file[5]
print(fname)

In [ ]:
d = sdf.read(fname)
fields = d.__dict__

In [ ]:
# look at all the data stored in the .sdf file
#list_sdf_variables(d)

In [ ]:
# normalization factor
norm = 2.73092449e-22

In [ ]:
# N_ele are the accelerated electrons, from the Nitrogen K-shell
# particle momenta
px = fields['Particles_Px_N_ele'].data / norm
py = fields['Particles_Py_N_ele'].data / norm
pz = fields['Particles_Pz_N_ele'].data / norm
# particle weights
w = fields['Particles_Weight_N_ele'].data
# particle positions
(x, y, z) = fields['Grid_Particles_N_ele'].data

In [ ]:
# get the grid
(xx, yy, zz) = fields['Grid_Grid_mid'].data
# get bounds
xx_min = xx.min()
xx_max = xx.max()
yy_min = yy.min()
yy_max = yy.max()
zz_min = zz.min()
zz_max = zz.max()

In [ ]:
mask = ((px > 2.) & ((py**2 + pz**2) / px**2 < 1.))
px, py, pz, w, x, y, z = [arr[mask] for arr in (px, py, pz, w, x, y, z)]

In [ ]:
ene = .511 * (np.sqrt(1. + px**2 + py**2 + pz**2) - 1.)

In [ ]:
H, edges = np.histogramdd(ene, bins=100, weights=w)

In [ ]:
import matplotlib as mpl

In [ ]:
# matplotlib parameters
mpl.rcParams.update({"axes.labelsize" : 22,
                                 "axes.titlesize" : 20,
                                 "font.size" : 18,
                                 "legend.fontsize" : 14,
                                 "axes.linewidth" : 1.5,
                                 "font.family" : "serif",
                                 "font.serif" : "Computer Modern Roman",
                                 "xtick.labelsize" : 20,
                                 "xtick.major.size" : 5.5,
                                 "xtick.major.width" : 1.5,
                                 "ytick.labelsize" : 20,
                                 "ytick.major.size" : 5.5,
                                 "ytick.major.width" : 1.5,
                                 "text.usetex" : True,
                                 "figure.autolayout" : True})

In [ ]:
height

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)
plot_kwds={'linewidth':5, 'alpha':0.5}
ax.semilogy(edges[0][:-1], H, basey=10, nonposy='mask', **plot_kwds)
ax.grid(color='gray', linestyle='--', linewidth=2)
ax.set_xlabel('Energy [MeV]')
ax.set_ylabel('Counts [Number per MeV]');
width  = 9.42
#height = width / 1.618
height = 4.989

fig.set_size_inches(width, height) 
fig.savefig("spectrum.pdf", transparent=True)

In [ ]:
dpi = 500
width = height = 9.333
area = width * height

In [ ]:
dpi * width

In [ ]:
angle = np.sqrt(py**2 + pz**2) / px

In [ ]:
H, edges = np.histogramdd(angle, bins=80, weights=w)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(edges[0][:-1], H, basey=10, nonposy='mask')

ax.set_xlabel(r'$\frac{p_{\perp}}{p_{\parallel}}$')
ax.set_ylabel('Counts [Number per 0.01 rad]');

In [ ]:
pperp = np.sqrt(py**2 + pz**2)

In [ ]:
H, edges = np.histogramdd([px, pperp], bins=100, weights=w)

In [ ]:
# replace zero values before taking the logarithm 
H[~(H > 0.)] = 1.

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(np.log10(H.T), origin='lower', interpolation='none',
                extent=np.array([edges[0].min(), edges[0].max(), edges[1].min(), edges[1].max()]),
                aspect='auto')
 
ax.set_xlabel(r'$p_{\parallel} [m_e c]$')
ax.set_ylabel(r'$p_{\perp} [m_e c]$')

cbar = colorbar(img)
cbar.set_label(r'$\log_{10}$' + ' Counts [Number per ' + r'$(m_e c)^2]$')

In [ ]:
H, edges = np.histogramdd([x, px], bins=100, weights=w)

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(H.T, origin='lower', interpolation='none',
                extent=np.array([edges[0].min(), edges[0].max(), edges[1].min(), edges[1].max()]),
                aspect='auto')
 
ax.set_xlabel(r'$x$' + ' [m]')
ax.set_ylabel(r'$p_x [m_e c]$')

cbar = colorbar(img)
cbar.set_label('Count [arb. unit]')

In [ ]:
def charge_ene_vs_x_3D_N(files):
    # normalization factor
    norm = 2.73092449e-22
    
    x_at_ene_max = list()
    ene_max = list()
    charge = list()
    
    for _, fname in files.items():
        print('processing {}'.format(fname))
        
        d = sdf.read(fname)
        fields = d.__dict__
        
        px = fields['Particles_Px_N_ele'].data / norm
        py = fields['Particles_Py_N_ele'].data / norm
        pz = fields['Particles_Pz_N_ele'].data / norm
        w = fields['Particles_Weight_N_ele'].data
        (x, _, _) = fields['Grid_Particles_N_ele'].data
        
        mask = (px > 30.)
        px, py, pz, w, x = [arr[mask] for arr in (px, py, pz, w, x)]
        
        ene = .511 * (np.sqrt(1. + px**2 + py**2 + pz**2) - 1.)
        
        charge.append(np.sum(w) * 1.6e-19)
        ene_max.append(np.max(ene))
        x_at_ene_max.append(x[np.argmax(ene)])

    return x_at_ene_max, ene_max, charge

In [ ]:
#t = get_sdf_files(rf)
t = OrderedDict([(5, '/home/berceanu/Development/pic/ming_jobs/ep_III3D1/0005.sdf'),
                 (9, '/home/berceanu/Development/pic/ming_jobs/ep_III3D1/0009.sdf')])

In [ ]:
x_at_ene_max, ene_max, charge = charge_ene_vs_x_3D_N(t)

In [ ]:
fig, ax1 = plt.subplots()

#left
ax1.scatter(x_at_ene_max, ene_max, c='b')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel(r'$E_{max}$ [MeV]', color='b')
ax1.set_xlabel('x [m]')
ax1.tick_params('y', colors='b')

#right
ax2 = ax1.twinx()
ax2.scatter(x_at_ene_max, np.array(charge)*1e9, c='r')
ax2.set_ylabel('Charge [nC]', color='r')
ax2.tick_params('y', colors='r')

fig.tight_layout()

In [ ]:
charge[0] * 1e+9

In [ ]:
# 3D visualization
import yt # more complex, higher quality

In [ ]:
#files = {10:'ep_III3D10/0009.sdf', 8:'ep_III3D8/0005.sdf', 3:'ep_III3D3/0008.sdf', 9:'ep_III3D9/0015.sdf', 0:'ep_III3D/0007.sdf',
         #2:'ep_III3D2/0008.sdf', 7:'ep_III3D7/0005.sdf', 5:'ep_III3D5/0016.sdf', 1:'ep_III3D1/0009.sdf', 6:'ep_III3D6/0020.sdf',
         #4:'ep_III3D4/0015.sdf'}

In [ ]:
#fname = os.path.join(root, files[8])
#print(fname)

In [ ]:
d = sdf.read(fname)
fields = d.__dict__

In [ ]:
list_sdf_variables(d)

In [ ]:
head = d.Header
for k, v in head.items():
    print(k, v)

In [ ]:
# N_ele are the accelerated electrons, from the Nitrogen K-shell
# particle momenta
px = fields['Particles_Px_N_ele'].data
py = fields['Particles_Py_N_ele'].data
pz = fields['Particles_Pz_N_ele'].data
# particle weights
w = fields['Particles_Weight_N_ele'].data
# particle positions
(x, y, z) = fields['Grid_Particles_N_ele'].data

In [ ]:
#string = 'Derived_Number_Density_N_ele'
#t = fields[string]
#t.units

In [ ]:
# load the electric field components
ex = fields['Electric_Field_Ex']
ey = fields['Electric_Field_Ey']
ez = fields['Electric_Field_Ez']

In [ ]:
# load the K-shell Nitrogen electrons
nNele = d.__dict__['Derived_Number_Density_N_ele']
rho = nNele.data

In [ ]:
# get the simulation box
(xx, yy, zz) = fields['Grid_Grid'].data
# get bounds
xx_min = xx.min()
xx_max = xx.max()
yy_min = yy.min()
yy_max = yy.max()
zz_min = zz.min()
zz_max = zz.max()
bbox = np.array([[xx_min, xx_max], [yy_min, yy_max], [zz_min, zz_max]])

In [ ]:
data = dict(number_density = (rho, '1/m**3'),
            particle_position_x = (x, 'm'), 
            particle_position_y = (y, 'm'),
            particle_position_z = (z, 'm'),
            particle_momentum_x = (px, 'kg*m/s'),
            particle_momentum_y = (py, 'kg*m/s'),
            particle_momentum_z = (pz, 'kg*m/s'),
            particle_weighting = (w, ''),
            electric_field_x = (ex.data, 'V/m'),
            electric_field_y = (ey.data, 'V/m'),
            electric_field_z = (ez.data, 'V/m')
           ) 

In [ ]:
ds = yt.load_uniform_grid(data, data['number_density'][0].shape, bbox=bbox, nprocs=4,
                          length_unit='m', mass_unit='kg', time_unit='s', velocity_unit='m/s', unit_system='mks')

In [ ]:
print(ds.domain_left_edge)
print(ds.domain_right_edge)
print(ds.domain_width)

In [ ]:
def _magnitude(field, data):
    return np.sqrt(data['electric_field_x']**2 + data['electric_field_y']**2 + data['electric_field_z']**2)

In [ ]:
ds.add_field(('stream', 'electric_field_magnitude'), units='V/m', function=_magnitude, sampling_type='cell')

In [ ]:
#ds.derived_field_list

In [ ]:
# plot the density, integrating over z
prj = yt.ProjectionPlot(ds, "z", 'number_density', origin='native', aspect=6/9)
prj.set_log("number_density", False);
#prj.annotate_particles((5e-6, 'm'), ptype='all', col='r', alpha=0.05)
#prj.set_axes_unit('m')
prj.set_buff_size(500)
prj.set_figure_size(5.8)
prj.set_font_size(22)
prj.save('density_z.png', mpl_kwargs={'facecolor': 'none', 'edgecolor': 'none', 'dpi': 500})

In [ ]:
#import yt.visualization.eps_writer as eps

In [ ]:
#eps_fig = eps.single_plot(prj)
#eps_fig.save_fig('zoom-pdf', format='pdf')

In [ ]:
#eps_fig = eps.multiplot_yt(1, 2, [prj, prj])
#eps_fig.save_fig('multi', format='pdf')

In [ ]:
# plot the density, integrating over x
prj = yt.ProjectionPlot(ds, "x", 'number_density', origin='native')
prj.set_log("number_density", False);
#prj.annotate_particles((5e-6, 'm'), ptype='all', col='r', alpha=0.05)
#prj.set_axes_unit('m')
prj.set_buff_size(500)
prj.set_figure_size(5.8)
prj.set_font_size(22)
prj.save('density_x.png', mpl_kwargs={'facecolor': 'none', 'edgecolor': 'none', 'dpi': 500})

In [ ]:
ppsx = yt.ParticlePlot(ds, 
                                         ('all', 'particle_position_x'), 
                                         ('all', 'particle_momentum_x'), 
                                         ('all', 'particle_weighting'),
                                         x_bins=800,
                                         y_bins=800)
#particle_phase_space_x.set_log('particle_weighting', False)
#ppsx.set_buff_size(500)
ppsx.set_unit('particle_position_x', 'um')
ppsx.set_ylabel('Momentum X')
ppsx.set_figure_size(5.8)
ppsx.set_font_size(22)
ppsx.save('particles_x.png', mpl_kwargs={'facecolor': 'none', 'edgecolor': 'none', 'dpi': 500})

ppsy = yt.ParticlePlot(ds, 
                                         ('all', 'particle_position_y'), 
                                         ('all', 'particle_momentum_y'), 
                                         ('all', 'particle_weighting'),
                                         x_bins=800,
                                         y_bins=800)
#particle_phase_space_y.set_log('particle_weighting', False);
#ppsy.set_buff_size(500)
ppsy.set_unit('particle_position_y', 'um')
ppsy.set_ylabel('Momentum Y')
ppsy.set_figure_size(5.8)
ppsy.set_font_size(22)
ppsy.save('particles_y.png', mpl_kwargs={'facecolor': 'none', 'edgecolor': 'none', 'dpi': 500})

In [ ]:
particle_phase_space_x

In [ ]:
particle_phase_space_y

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid

fig = plt.figure()

# See http://matplotlib.org/mpl_toolkits/axes_grid/api/axes_grid_api.html
# These choices of keyword arguments produce a four panel plot that includes
# four narrow colorbars, one for each plot.  Axes labels are only drawn on the
# bottom left hand plot to avoid repeating information and make the plot less
# cluttered.
grid = AxesGrid(fig, (0.075,0.075,0.85,0.85),
                nrows_ncols = (2, 2),
                axes_pad = 1.0,
                label_mode = "1",
                share_all = True,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="3%",
                cbar_pad="0%")

fields = ['electric_field_x', 'electric_field_y', 'electric_field_z', 'electric_field_magnitude']

# Create the plot.  Since SlicePlot accepts a list of fields, we need only
# do this once.
p = yt.SlicePlot(ds, 'z', fields, origin='native', aspect=6/9)
p.set_axes_unit('m')

# Velocity is going to be both positive and negative, so let's make these
# slices use a linear colorbar scale
p.set_log('electric_field_x', False)
p.set_log('electric_field_y', False)
p.set_log('electric_field_z', False)
# p.annotate_streamlines('electric_field_x', 'electric_field_y')
# p.set_log('electric_field_magnitude', False)

# p.zoom(2)

# For each plotted field, force the SlicePlot to redraw itself onto the AxesGrid
# axes.
for i, field in enumerate(fields):
    plot = p.plots[field]
    plot.figure = fig
    plot.axes = grid[i].axes
    plot.cax = grid.cbar_axes[i]

# Finally, redraw the plot on the AxesGrid axes.
p._setup_plots()

plt.savefig('multiplot_2x2.png')

In [ ]:
slc = yt.SlicePlot(ds, 'z', 'electric_field_magnitude', origin='native', aspect=6/20)
#slc.set_axes_unit('m')   
#slc.annotate_streamlines('electric_field_x', 'electric_field_y')
slc.annotate_line_integral_convolution('electric_field_x', 'electric_field_y', lim=(0.4,0.65))
#slc.set_log('electric_field_magnitude', False)
slc.set_figure_size(6.8)
slc.set_font_size(20)
slc.save('magnitude.png', mpl_kwargs={'facecolor': 'none', 'edgecolor': 'none', 'dpi': 500})

In [ ]:
slc

In [ ]:
# volume rendering

In [ ]:
# for loading images from disk into the notebook
from IPython.display import Image

In [ ]:
xx_cent = center_linspace(xx)
xx_cent_min = xx_cent.min()
xx_cent_max = xx_cent.max()
bbox = np.array([[xx_cent_min, xx_cent_max], [yy_min, yy_max], [zz_min, zz_max]])

In [ ]:
data = dict(number_density = (rho, '1/m**3'),
            electric_field_x = (ex.data, 'V/m'),
            electric_field_y = (ey.data, 'V/m'),
            electric_field_z = (ez.data, 'V/m')
           ) 

In [ ]:
ds = yt.load_uniform_grid(data, data['number_density'][0].shape, length_unit='m', unit_system='mks',
                          bbox=bbox, nprocs=4)

In [ ]:
print(ds.domain_left_edge)
print(ds.domain_right_edge)
print(ds.domain_width)

In [ ]:
# turn the original 3D array into 1D
flat_n = rho.flatten()
# fix negative logarith values
flat_n[flat_n < 1.] = 1.

# histogram of the logarithm of n_{N_{ele}}
fig, ax = plt.subplots()
ax.hist(np.log10(flat_n), bins='auto', range=(18, 25), fc='gray', histtype='stepfilled', alpha=0.3, normed=True);
ax.set_ylabel('Density function')
ax.set_xlabel(r'$\log_{10}(n_{N_{ele}})$');

In [ ]:
sc = yt.create_scene(ds, 'number_density', lens_type='perspective')

# Get a reference to the VolumeSource associated with this scene
# It is the first source associated with the scene, so we can refer to it
# using index 0.
source = sc[0]

# Set the bounds of the transfer function
source.tfh.set_bounds((1e21, 1e24))

# set that the transfer function should be evaluated in log space
source.tfh.set_log(True)

# Do not make underdense regions appear opaque
source.tfh.grey_opacity = False

# Plot the transfer function, along with the CDF of the density field to
# see how the transfer function corresponds to structure in the CDF
source.tfh.plot('transfer_function.png', profile_field='number_density')

sc.annotate_axes(alpha=0.5)

cam = sc.add_camera(ds, lens_type='perspective')
cam.rotate(np.pi/6, rot_center=np.array([0.0, 0.0, 0.0]))
cam.zoom(1.5)
cam.resolution = (4666, 4666)

#sc.annotate_domain(ds, color=[1, 1, 1, 1])


# save the image, flooring especially bright pixels for better contrast
sc.save('nNele.png', sigma_clip=3.0)

In [ ]:
Image(filename='nNele.png', width=512, height=512)

In [ ]:
import ipyvolume as ipv # simple, decent

In [ ]:
ipv.quickvolshow(rho.T, lighting=True)

In [ ]:
Image(filename='transfer_function.png')

Unused code follows.

In [ ]:
exdata = ex.data[:,:,ex.data.shape[-1]//2]
psi = np.zeros_like(exdata)

for i in range(psi.shape[0]-2, -1, -1):
    psi[i,:] = psi[i + 1,:] + exdata[i, :]

dx = xx[1] - xx[0]
psi = psi * dx / 510998.946

In [ ]:
# wake pseudo potential
fig, ax = plt.subplots()
img = ax.imshow(psi.T, origin='lower', interpolation='none',
                extent=np.array([xx.min(), xx.max(), yy.min(), yy.max()]),
                aspect='auto')
 
ax.set_xlabel(ex.grid_mid.labels[0] + ' [m]', labelpad=-1)
ax.set_ylabel(ex.grid_mid.labels[1] + ' [m]', labelpad=0)

cbar = colorbar(img)
cbar.set_label(r'$\Psi$')

In [ ]:
squeezed_data = np.squeeze(np.sum(np.abs(ey.data),axis=0))

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(squeezed_data.T, origin='lower', interpolation='none',
                extent=np.array([yy.min(), yy.max(), zz.min(), zz.max()]),
                aspect='auto')
 
ax.set_xlabel(ey.grid_mid.labels[1] + ' [m]', labelpad=-1)
ax.set_ylabel(ey.grid_mid.labels[2] + ' [m]', labelpad=0)

cbar = colorbar(img)
cbar.set_label(ey.name + r' $(' + ey.units + r')$')

In [ ]:
fig, ax = plt.subplots()
ax.plot(yy, squeezed_data[:, ey.data.shape[-2]//2])
ax.set_xlabel(ey.grid_mid.labels[1] + ' [m]', labelpad=0)
ax.set_ylabel(ey.name + r' $(' + ey.units + r')$');

In [ ]:
def _abs(field, data):
    return np.abs(data['electric_field_y'])

In [ ]:
ds.add_field(('stream', 'electric_field_y_abs'), units='V/m', function=_abs, sampling_type='cell')

In [ ]:
proj = ds.proj('electric_field_y_abs', 'x')
frb = proj.to_frb(ds.domain_width[1], ey.data.shape[1])
squeezed_data = frb['electric_field_y_abs']

In [ ]:
ad = ds.all_data()
phaseplot_z = yt.ParticlePhasePlot(ad, 
                                   ('all', 'particle_position_x'), 
                                   ('all', 'particle_position_y'), 
                                   ('all', 'particle_weighting'))

In [ ]:
phaseplot_z